In [2]:
# ================================================
# 📌 1. Import Libraries
# ================================================
%pip install cassandra-driver tensorflow scikit-learn plotly --quiet

import pandas as pd
import numpy as np
from cassandra.cluster import Cluster
from datetime import datetime, timedelta
from scipy.signal import find_peaks

# Deep Learning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import train_test_split

# Visualization
import plotly.graph_objects as go


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# ================================================
try:
    cluster = Cluster(['127.0.0.1'], port=9042)  # แก้ตาม Cassandra ของคุณ
    session = cluster.connect('stock_data')

    stock = "AOT"
    start_date = datetime.now() - timedelta(days=150)

    rows = session.execute(f"""
        SELECT * FROM candlestick_data 
        WHERE symbol = '{stock}'
    """)
    data = pd.DataFrame(rows)

    if data.empty:
        raise ValueError("❌ No data found")
except Exception as e:
    print(f"❌ Cassandra connection or query failed: {e}")
    # Create dummy data for demonstration if Cassandra is not available
    dates = pd.date_range(datetime.now() - timedelta(days=150), periods=200)
    np.random.seed(42)
    prices = np.cumsum(np.random.randn(200)) + 70
    data = pd.DataFrame({
        'time': dates,
        'open_price': prices + np.random.uniform(-1, 1, size=200),
        'high_price': prices + np.random.uniform(0, 2, size=200),
        'low_price': prices - np.random.uniform(0, 2, size=200),
        'close_price': prices + np.random.uniform(-1, 1, size=200),
    })
    stock = "AOT"
    start_date = data['time'].min()
if data.empty:
    raise ValueError("❌ No data found")

# จัดรูปแบบ DataFrame
data['time'] = pd.to_datetime(data['time'])
data = data[data['time'] >= start_date]
data = data.sort_values('time').reset_index(drop=True)



WARNING	cassandra.cluster:cluster.py:__init__()- Cluster.__init__ called with contact_points specified, but no load_balancing_policy. In the next major version, this will raise an error; please specify a load-balancing policy. (contact_points = ['127.0.0.1'], lbp = None)
WARNING	cassandra.cluster:cluster.py:_reconnect_internal()- [control connection] Error connecting to 127.0.0.1:9042:
 Traceback (most recent call last):
   File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/cassandra/cluster.py", line 3577, in _reconnect_internal
    return self._try_connect(host)
           ^^^^^^^^^^^^^^^^^^^^^^^
   File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/cassandra/cluster.py", line 3599, in _try_connect
    connection = self._cluster.connection_factory(host.endpoint, is_control_connection=True)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
   File "/Library/Frameworks/Pytho

❌ Cassandra connection or query failed: ('Unable to connect to any servers', {'127.0.0.1:9042': ConnectionRefusedError(61, "Tried connecting to [('127.0.0.1', 9042)]. Last error: Connection refused")})


In [4]:
# ================================================
# 📌 3. Head & Shoulders Detection (Rule-based)
# ================================================
def detect_head_and_shoulders(prices, distance=3, tolerance=0.05):
    """Return True if pattern exists in window"""
    peaks, _ = find_peaks(prices, distance=distance)
    if len(peaks) < 3:
        return False
    for i in range(len(peaks) - 2):
        ls, head, rs = peaks[i], peaks[i+1], peaks[i+2]
        if head > rs or ls > head: 
            continue
        ls_val, head_val, rs_val = prices[ls], prices[head], prices[rs]
        if head_val > ls_val and head_val > rs_val:
            if abs(ls_val - rs_val) / max(ls_val, rs_val) < tolerance:
                return True
    return False



In [5]:
# ================================================
# 📌 4. Generate Dataset for CNN
# ================================================
window_size = 40
X, y = [], []
close_prices = data['close_price'].values

for i in range(len(close_prices) - window_size):
    window = close_prices[i:i+window_size]
    label = 1 if detect_head_and_shoulders(window) else 0
    X.append(window)
    y.append(label)

X = np.array(X)
y = np.array(y)

# Reshape for Conv1D input (samples, timesteps, features)
X = X.reshape(-1, window_size, 1)

In [6]:

# ================================================
# 📌 5. Train/Test Split
# ================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [7]:
# ================================================
# 📌 6. Build CNN Model
# ================================================
model = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(window_size, 1)),
    MaxPooling1D(2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 38, 32)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 19, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 608)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │        38,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 39,169 (153.00 KB)

 Trainable params: 39,169 (153.00 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# ================================================
# 📌 7. Train Model
# ================================================
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.3296 - loss: 5.5122 - val_accuracy: 0.8077 - val_loss: 2.5486
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8412 - loss: 2.3067 - val_accuracy: 0.8077 - val_loss: 3.3125
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8589 - loss: 2.5195 - val_accuracy: 0.8077 - val_loss: 3.1018
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8401 - loss: 2.5348 - val_accuracy: 0.8077 - val_loss: 2.2645
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8474 - loss: 1.6726 - val_accuracy: 0.8077 - val_loss: 1.2602
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8651 - loss: 0.7171 - val_accuracy: 0.8077 - val_loss: 0.4989
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7043 - loss: 0.5652 - val_accuracy: 0.8077 - val_loss: 0.6125
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8938 - loss: 0.5102 - val_accuracy: 0.8077 - val_loss: 0.7198


In [9]:

# ================================================
# 📌 8. Evaluate Model
# ================================================
loss, acc = model.evaluate(X_test, y_test)
print(f"\n✅ Test Accuracy: {acc:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7812 - loss: 0.5003

✅ Test Accuracy: 0.78


In [10]:

# ================================================
# 📌 9. Find HNS Patterns for Visualization
# ================================================
def find_hns_patterns(prices, distance=3, tolerance=0.05):
    """Return list of (left, head, right) indices if found"""
    peaks, _ = find_peaks(prices, distance=distance)
    patterns = []
    if len(peaks) < 3:
        return patterns
    for i in range(len(peaks) - 2):
        ls, head, rs = peaks[i], peaks[i+1], peaks[i+2]
        if head > rs or ls > head: 
            continue
        ls_val, head_val, rs_val = prices[ls], prices[head], prices[rs]
        if head_val > ls_val and head_val > rs_val:
            if abs(ls_val - rs_val) / max(ls_val, rs_val) < tolerance:
                patterns.append((ls, head, rs))
    return patterns




In [11]:
# ================================================
# 📌 10. Plot Candlestick with Detected Patterns
# ================================================
def plot_chart_with_pattern(df, pattern_points, ticker):
    fig = go.Figure()

    # Candlestick chart
    fig.add_trace(go.Candlestick(
        x=df['time'],
        open=df['open_price'],
        high=df['high_price'],
        low=df['low_price'],
        close=df['close_price'],
        name='Candlestick'
    ))

    # วาดจุด Head & Shoulders
    for (l, h, r) in pattern_points:
        fig.add_trace(go.Scatter(
            x=[df['time'].iloc[l]], y=[df['close_price'].iloc[l]],
            mode='markers+text', name='Left Shoulder',
            marker=dict(color='yellow', size=10),
            text=["Left"], textposition="top center"
        ))
        fig.add_trace(go.Scatter(
            x=[df['time'].iloc[h]], y=[df['close_price'].iloc[h]],
            mode='markers+text', name='Head',
            marker=dict(color='red', size=12),
            text=["Head"], textposition="top center"
        ))
        fig.add_trace(go.Scatter(
            x=[df['time'].iloc[r]], y=[df['close_price'].iloc[r]],
            mode='markers+text', name='Right Shoulder',
            marker=dict(color='green', size=10),
            text=["Right"], textposition="top center"
        ))

        # neckline
        fig.add_shape(
            type='line',
            x0=df['time'].iloc[l], y0=df['close_price'].iloc[l],
            x1=df['time'].iloc[r], y1=df['close_price'].iloc[r],
            line=dict(color='green', dash='dash')
        )

    fig.update_layout(
        title=f'{ticker} - Head and Shoulders Pattern',
        xaxis_title='Date',
        yaxis_title='Price (Baht)',
        template='plotly_white',
        xaxis_rangeslider_visible=False,
        height=600
    )
    fig.show()



In [12]:
# ================================================
# 📌 11. Run Detection + Plot
# ================================================
patterns = find_hns_patterns(data['close_price'].values)
plot_chart_with_pattern(data, patterns, stock)